In [19]:
import numpy as np
import pickle
import matplotlib.pyplot as plt
import copy

import random


In [2]:
corpus = []
f = open('alice_in_wonderland.txt','r')
while(1):
    line =  f.readline()
    if len(line) == 0: break
    corpus.extend(line.split())
        
f.close()
corpus = ' '.join(corpus)

def clean_word(word):
    word = word.lower()
    for punctuation in ['"',"'",'.',',','-','?','!',';',':','—','(',')','[',']']:
        word = word.split(punctuation)[0]
    return word



corpus = [clean_word(word) for word in corpus.split()]
corpus = [word for word in corpus if len(word) > 0]
print(corpus[:25])
D = len(corpus)
print('corpus len: ',D)

['alice', 'adventures', 'in', 'wonderland', 'by', 'lewis', 'carroll', 'the', 'millennium', 'fulcrum', 'edition', '3', 'contents', 'chapter', 'i', 'down', 'the', 'rabbit', 'chapter', 'ii', 'the', 'pool', 'of', 'tears', 'chapter']
corpus len:  25320


In [3]:
tokenize = {}
wordlist = []
token = 0
for word in corpus:
    if word not in tokenize.keys():
        tokenize[word] = token
        wordlist.append(word)
        token += 1
    
V = len(wordlist)
print('word list size (number of distinct words): ', V)

word list size (number of distinct words):  2501


In [4]:
# bin how many times a word follows another word
counts_2gram = np.zeros((V,V))
for i in range(1,len(corpus)):
    token_i = tokenize[corpus[i]]
    token_im1 = tokenize[corpus[i-1]]
    counts_2gram[token_i,token_im1] += 1
print(counts_2gram)
    

[[0. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [9. 1. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [5]:
#past word as feature

posterior_1word = np.zeros((V, V))
prior = np.zeros(V)

def fill_prior():
    map = {}
    for word in corpus:
        if word in map:
            map[word] += 1
        else:
            map[word] = 1
    
    index = 0     
    for word in wordlist:
        token = tokenize[word]
        prior[token] = map[word]/len(corpus)
        


def fill_in_posterior_1word():
    for word in wordlist:
        token = tokenize[word]
        wordArray = counts_2gram[token]
            
        total = 0
        for count in wordArray:
            total += count
                
        newArray = []
            
        for count in wordArray:
            newArray.append(count / total)
            
        posterior_1word[token] = newArray  
        
fill_in_posterior_1word()
fill_prior()
    
def pred_2gram(word):
    token = tokenize[word]
    
    array = posterior_1word[token]
    
    index = 0
    value = 0
    for i in range(len(array)):
        if array[i] > value:
            value = array[i]
            index = i
    ## print(value)
    
    print(wordlist[index])
    return posterior_1word[token][index] * prior[index]  
    
print(pred_2gram('alice'))
print(pred_2gram('the'))
print(pred_2gram('cat'))
print(pred_2gram('turtle'))

said
0.005374061880700568
said
0.0023591151671500023
the
0.03960014516886554
mock
0.0021748734838139606


In [8]:
## posterior_1word = np.zeros((V, V))

def whole_corpus_2gram():
    index = 0
    count = 0
    for word in corpus:
        prevToken = tokenize[word]
        
        if index >= len(corpus) - 1:
            break
        
        newToken = 0
        value = -1
        for i in range(V):
            temp = posterior_1word[i][prevToken]
            temp *= prior[i]
            if temp > value:
                value = temp
                newToken = i
                
        
            
        newWord = wordlist[newToken]
        if newWord == corpus[index + 1]:
            count += 1
        
        index += 1
    return count / (index - 1)

print(whole_corpus_2gram())

0.24535903309898097


In [6]:
def create_posterior(size):
    posterior_word = np.zeros((size,V, V))
    
    for k in range(size):
        counts_gram = np.zeros((V,V))
        for i in range(k+1,len(corpus)):
            token_i = tokenize[corpus[i]]
            
            
            for j in range(i-k-1, i):
                token_im1 = tokenize[corpus[j]]
                counts_gram[token_i,token_im1] += 1
        
                
        posterior_word_temp = np.zeros((V,V))
        
        for word in wordlist:
            token = tokenize[word]
            wordArray = counts_gram[token]
            
            total = 0
            for count in wordArray:
                total += count
            newArray = []
            for count in wordArray:
                if total == 0:
                    newArray.append(0)
                else:
                    newArray.append(count / total)

            posterior_word_temp[token] = newArray 
            
        posterior_word[k] = posterior_word_temp
    return posterior_word
        

In [11]:
#past k words as features
posterior_word_copy = create_posterior(4)
def pred_word(array):
    
    main_array = np.ones(V)
    
    for i in range(len(array)):
        posterior_word_temp = posterior_word_copy[i]
        word = array[len(array) - 1 - i]
        token = tokenize[word]
         
        for j in range(V):
            main_array[j] *= posterior_word_temp[j][token]
           
    
    finalToken = 0
    value = -1
    for i in range(len(main_array)):
        if (main_array[i] * prior[i] > value):
            
            value = main_array[i] * prior[finalToken]
            finalToken = i
    ## print(value)
    return wordlist[finalToken]

def pred_word_list_array(array):
    
    main_array = np.ones(V)
    
    for i in range(len(array)):
        posterior_word_temp = posterior_word_copy[i]
        word = array[len(array) - 1 - i]
        token = tokenize[word]
         
        for j in range(V):
            main_array[j] *= posterior_word_temp[j][token]
           
    
    finalToken = 0
    value = -1
    for i in range(len(main_array)):
        main_array[i] = main_array[i] * prior[i]

    return main_array
    
    
## print(pred_word(['what','an','ignorant','little']))

In [9]:
posterior_word_copy = create_posterior(2)

def whole_corpus_gram(arg):
    index = 0
    count = 0
 
    word_arr = []
    
    for word in corpus:
        
        prevToken = tokenize[word]
        
        word_arr.append(word)
        
        if (len(word_arr) == arg):
            if index >= len(corpus) - 1:
                break

            newToken = 0
            value = -1

            predicted_word = pred_word(word_arr)

            if predicted_word == corpus[index + 1]:
                count += 1
            word_arr.pop(0)

            
        
        index += 1
        
     
    return count / (index - 1)

print(whole_corpus_gram(2))

0.47337862390394186


In [10]:
posterior_word_copy = create_posterior(4)

def whole_corpus_gram(arg):
    index = 0
    count = 0
 
    word_arr = []
    
    for word in corpus:
        
        prevToken = tokenize[word]
        
        word_arr.append(word)
        
        if (len(word_arr) == arg):
            if index >= len(corpus) - 1:
                break

            newToken = 0
            value = -1

            predicted_word = pred_word(word_arr)

            if predicted_word == corpus[index + 1]:
                count += 1
            word_arr.pop(0)

            
        
        index += 1
        
     
    return count / (index - 1)

print(whole_corpus_gram(4))

0.7869104984595939


In [10]:
posterior_word_copy = create_posterior(9)

def whole_corpus_gram(arg):
    index = 0
    count = 0
 
    word_arr = []
    
    for word in corpus:
        
        prevToken = tokenize[word]
        
        word_arr.append(word)
        
        if (len(word_arr) == arg):
            if index >= len(corpus) - 1:
                break

            newToken = 0
            value = -1

            predicted_word = pred_word(word_arr)

            if predicted_word == corpus[index + 1]:
                count += 1
            word_arr.pop(0)

            
        
        index += 1
        
     
    return count / (index - 1)

print(whole_corpus_gram(9))

0.908879058377439


In [12]:
posterior_word_copy = create_posterior(3)

random_arr = ['the', 'mad', 'hatter']

for i in range(25):
    pred_next_word = pred_word(random_arr)
    
    print(pred_next_word)
    
    random_arr.pop(0)
    random_arr.append(pred_next_word)
    


with
this
a
sudden
alice
but
very
glad
they
get
to
her
eyes
still
it
was
very
provoking
to
find
that
her
neck
as
that


In [21]:
posterior_word_copy = create_posterior(3)
random_arr = ['the', 'mad', 'hatter']

for i in range(25):
    pred_next_word_array = pred_word_list_array(random_arr)
    random_str = random.choices(wordlist, weights=pred_next_word_array, k=1)
    
    print(random_str[0])
    
    random_arr.pop(0)
    random_arr.append(random_str[0])

said
alice
replied
rather
shyly
hardly
know
sir
just
at
present
least
least
least
least
notice
of
her
going
though
as
she
was
out
exactly
